# __*Segmenting and Clustering Neighborhoods in Toronto*__

 Author : Abhijeet Prasad

In [103]:
# Install bs4 library
!pip install bs4

In [104]:
# Import libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

## __Importing Data__

In [105]:
# get the contents of the webpage in text format and store in a variable called data
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data  = requests.get(url).text

In [106]:
soup = BeautifulSoup(data,"html5lib")

In [107]:
#find a html table in the web page
tables = soup.find_all('table')
len(tables) # Total Number of tables in the file

3

In [108]:
toronto_data = pd.DataFrame(columns=["Postal Code", "Borough", "Neighborhood"])

for row in tables[0].findAll('td'):      # tables[0] contains the data we are intrested in.
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        Postal_Code = row.p.text[:3]
        borough = (row.span.text).split('(')[0]
        neighborhood = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        toronto_data = toronto_data.append({"Postal Code":Postal_Code, "Borough":borough, "Neighborhood":neighborhood}, ignore_index=True)

toronto_data['Borough']=toronto_data['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

toronto_data

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [109]:
toronto_data.shape

(103, 3)

## __Getting Latitude and Longitude coordinates of each neighborhood__

In [110]:
# install gecoder library
!pip install geocoder

__The Below Code doesn't work__

__Alternate Method- Using the CSV file Provided__

In [111]:
coordinates = pd.read_csv("https://cocl.us/Geospatial_data")
coordinates

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


### __Adding Latitude and Longitude to `neighborhood_data` dataframe__

In [112]:
coordinates.shape

(103, 3)

In [113]:
# Since the dimensions are the same, we can try to join on the postal codes to get the required data.

toronto_data['Latitude']=coordinates['Latitude']
toronto_data['Longitude']=coordinates['Longitude']

toronto_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Queen's Park,Ontario Provincial Government,43.773136,-79.239476
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.706876,-79.518188
99,M4Y,Downtown Toronto,Church and Wellesley,43.696319,-79.532242
100,M7Y,East Toronto Business,Enclave of M4L,43.688905,-79.554724
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.739416,-79.588437


In [114]:
toronto_data.shape

(103, 5)

# __Cluster Toronto Neighborhood based on Venue Categories__

We cluster Toronto based on the similarities of the venues categories using Kmeans clustering and Foursquare API.

In [115]:
!pip install geopy

In [116]:
from geopy.geocoders import Nominatim
import folium

_Let's get the geographical coordinates of Manhattan._

In [117]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### _Let's visualize Toronto and the neighborhoods in it._

In [118]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### _Next, we use the Foursquare API to explore the neighborhoods of Toronto and segment them._

In [119]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'NAMVCN3ZZZ2IIBILNNZ2MVA2LL313ZCWBCZTYJYUQXAK3UIF' # your Foursquare ID
CLIENT_SECRET = 'BNS3DBVVJJLQKFS3KUDPE1KSHKTWQXJLI0RVHDJHWDKIH40D' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

#### Creating a function to get all the venue categories from all the neighborhoods in Toronto


In [120]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [121]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'], latitudes=toronto_data['Latitude'],
                                 longitudes=toronto_data['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [122]:
# check the size of the resulting dataframe
print(toronto_venues.shape)
toronto_venues.head()

(2000, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Victoria Village,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Regent Park, Harbourfront",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,"Regent Park, Harbourfront",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Regent Park, Harbourfront",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


### Checking venues returned for each neighborhood in Toronto

In [123]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,23,23,23,23,23,23
"Bathurst Manor, Wilson Heights, Downsview North",19,19,19,19,19,19
Bayview Village,20,20,20,20,20,20
"Bedford Park, Lawrence Manor East",81,81,81,81,81,81
Berczy Park,1,1,1,1,1,1
...,...,...,...,...,...,...
"Willowdale, Newtonbrook",69,69,69,69,69,69
Woburn,29,29,29,29,29,29
Woodbine Heights,3,3,3,3,3,3


## __Analyzing Each Neighborhood__


In [124]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [125]:
toronto_onehot.shape

(2000, 260)

#### Next, we group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [126]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0
1,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0
2,Bayview Village,0.050000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.05,0.000000,0.000000,0.0,0.0
3,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.012346,0.000000,0.000000,0.00,0.012346,0.000000,0.0,0.0
4,Berczy Park,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,"Willowdale, Newtonbrook",0.014493,0.0,0.014493,0.014493,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.014493,0.0,0.0
97,Woburn,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.034483,0.00,0.000000,0.000000,0.0,0.0
98,Woodbine Heights,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0
99,York Mills West,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.029412,0.000000,0.00,0.000000,0.000000,0.0,0.0


#### A function to sort the venues in descending order.

In [127]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create a new dataframe and display the top 10 venues for each neighborhood.

In [128]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Sandwich Place,Café,Bakery,Coffee Shop,Nightclub,Restaurant,Stadium,Intersection,Italian Restaurant
1,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Gift Shop,Health Food Store,Shopping Mall,Sandwich Place,Diner,Bridal Shop,Intersection,Ice Cream Shop
2,Bayview Village,Indian Restaurant,Yoga Studio,Park,Coffee Shop,Restaurant,Burger Joint,Intersection,Bank,Fast Food Restaurant,Supermarket
3,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Cocktail Bar,Café,Clothing Store,Restaurant,Farmers Market,Japanese Restaurant,Gym,Gastropub
4,Berczy Park,Cafeteria,Women's Store,Cuban Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


### __Cluster Neighborhood__

In [129]:
from sklearn.cluster import KMeans

#### k-means to cluster the neighborhood into 4 clusters.

In [130]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 2, 0], dtype=int32)

#### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [131]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.dropna(axis=0, how='any', inplace=True)

toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.806686,-79.194353,0.0,Fast Food Restaurant,Women's Store,Electronics Store,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
1,M4A,North York,Victoria Village,43.784535,-79.160497,0.0,Bar,Women's Store,Cupcake Shop,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711,0.0,Donut Shop,Bank,Electronics Store,Restaurant,Mexican Restaurant,Medical Center,Breakfast Spot,Rental Car Location,Intersection,Deli / Bodega
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917,0.0,Coffee Shop,Korean BBQ Restaurant,Soccer Field,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Cupcake Shop,Discount Store
4,M7A,Queen's Park,Ontario Provincial Government,43.773136,-79.239476,0.0,Athletics & Sports,Gas Station,Bakery,Bank,Thai Restaurant,Fried Chicken Joint,Caribbean Restaurant,Hakka Restaurant,Distribution Center,Discount Store


### Finally, let's visualize the resulting clusters

In [133]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [134]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(int(cluster)), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## __Verifying each cluster__

#### Cluster 1

In [136]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0.0,Fast Food Restaurant,Women's Store,Electronics Store,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
1,North York,0.0,Bar,Women's Store,Cupcake Shop,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
2,Downtown Toronto,0.0,Donut Shop,Bank,Electronics Store,Restaurant,Mexican Restaurant,Medical Center,Breakfast Spot,Rental Car Location,Intersection,Deli / Bodega
3,North York,0.0,Coffee Shop,Korean BBQ Restaurant,Soccer Field,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Cupcake Shop,Discount Store
4,Queen's Park,0.0,Athletics & Sports,Gas Station,Bakery,Bank,Thai Restaurant,Fried Chicken Joint,Caribbean Restaurant,Hakka Restaurant,Distribution Center,Discount Store
...,...,...,...,...,...,...,...,...,...,...,...,...
96,Downtown Toronto,0.0,Gym,Furniture / Home Store,Women's Store,Diner,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store
99,Downtown Toronto,0.0,Pizza Place,Coffee Shop,Discount Store,Sandwich Place,Intersection,Playground,Middle Eastern Restaurant,Chinese Restaurant,College Rec Center,Cupcake Shop
100,East Toronto Business,0.0,Bus Line,Pizza Place,Park,Sandwich Place,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Women's Store
101,Etobicoke,0.0,Grocery Store,Coffee Shop,Pharmacy,Indie Movie Theater,Fast Food Restaurant,Pizza Place,Sandwich Place,Beer Store,Discount Store,Diner


### Cluster 2

In [137]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Downtown Toronto,1.0,Baseball Field,Women's Store,Cupcake Shop,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
97,Downtown Toronto,1.0,Baseball Field,Paper / Office Supplies Store,Discount Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Women's Store


### Cluster 3

In [139]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,East York,2.0,Playground,Park,Intersection,Women's Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
21,York,2.0,Park,Women's Store,Cuban Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
23,East York,2.0,Park,Convenience Store,Women's Store,Diner,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store
25,Downtown Toronto,2.0,Fast Food Restaurant,Park,Food & Drink Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Cuban Restaurant,Diner
40,North York,2.0,Park,Convenience Store,Home Service,Women's Store,Dim Sum Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner
50,North York,2.0,Park,Trail,Playground,Women's Store,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
74,Central Toronto,2.0,Park,Women's Store,Pool,Dim Sum Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner
90,Scarborough,2.0,River,Park,Women's Store,Dim Sum Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner
98,Etobicoke,2.0,Jewelry Store,Park,Discount Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Women's Store


### Cluster 4

In [140]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,3.0,Playground,Women's Store,Cuban Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


### We have successfully cluster Toronto Neighborhood based on venue categories!